In [1]:
!pip install confluent_kafka
!pip install pymongo


In [2]:
from confluent_kafka import Consumer
import pymongo

In [3]:
def login_info():
    info={
        'bootstrap.servers':'KAFKA SERVER URL',
        'security.protocol':'SASL_SSL',
        'sasl.mechanism':'PLAIN',
        'sasl.username':'KAFKA SERVER USERNAME',
        'sasl.password':'KAFKA SERVER PASSWORD',
        'partitioner':'consistent_random',
        'group.id':'testing',
        'auto.offset.reset':'earliest'

    }
    return info

In [4]:
c=Consumer(login_info())

In [5]:
print(c.list_topics().topics)
d=c.list_topics().topics

{'Case': TopicMetadata(Case, 6 partitions), 'Region': TopicMetadata(Region, 6 partitions), 'test_topic': TopicMetadata(test_topic, 6 partitions), 'TimeProvince': TopicMetadata(TimeProvince, 6 partitions)}


In [6]:
topic='TimeProvince'
c.subscribe([topic])

In [7]:
client=pymongo.MongoClient('MONGO DB URL')

In [8]:
Database='Kafka'
collection='TimeProvince'

In [9]:
db=client[Database]
col=db[collection]

In [10]:
while True:
    try:
        msg=c.poll(5.0)
        
        if msg is None:
            print('wainting for messages!!')
            continue
        elif msg is not None:
            msg_value=msg.value().decode('utf-8')
            #print(msg_value)
            print(type(msg_value))
            y=eval(msg_value)
            t=col.insert_one(y)
            print(f'The following messages are f{msg.value()} which was delivered')
            agg = col.aggregate([
            {
                '$group': {
                '_id': None,
                'data': { '$push': '$$ROOT' }
                }
            },
            {    
                '$unwind': '$data'
            },
            {
                '$replaceRoot': { 'newRoot': '$data' }
            },
            {
                '$project': {
                '_id': 0
                }
            },
            {
                '$out': 'final_timeprovince'
            }
                        ])
            
    except KeyboardInterrupt:
        print('closing the consumer code')
        break


c.close()

<class 'str'>
The following messages are fb"{'date': '2020-01-20', 'time': 16, 'province': 'Gyeongsangbuk-do', 'confirmed': 0, 'released': 0, 'deceased': 0}" which was delivered
<class 'str'>
The following messages are fb"{'date': '2020-01-21', 'time': 16, 'province': 'Seoul', 'confirmed': 0, 'released': 0, 'deceased': 0}" which was delivered
<class 'str'>
The following messages are fb"{'date': '2020-01-21', 'time': 16, 'province': 'Daejeon', 'confirmed': 0, 'released': 0, 'deceased': 0}" which was delivered
<class 'str'>
The following messages are fb"{'date': '2020-01-21', 'time': 16, 'province': 'Gyeonggi-do', 'confirmed': 0, 'released': 0, 'deceased': 0}" which was delivered
<class 'str'>
The following messages are fb"{'date': '2020-01-21', 'time': 16, 'province': 'Chungcheongnam-do', 'confirmed': 0, 'released': 0, 'deceased': 0}" which was delivered
<class 'str'>
The following messages are fb"{'date': '2020-01-21', 'time': 16, 'province': 'Jeollabuk-do', 'confirmed': 0, 'released':